In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#/content/drive/MyDrive/Study/Big Data/Assignment10

In [2]:
!pip install pyspark

     |████████████████████████████████| 212.4MB 73kB/s 
     |████████████████████████████████| 204kB 19.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=ec72a063f36d4a18bacc82528ccfcb41fd7262e3dcfa67a3f138c4f0e703fad9
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


# Temp

In [3]:
!sudo apt-get install Netcat
!apt-get -q -y install netcat-openbsd

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package Netcat
Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  netcat-openbsd
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 39.8 kB of archives.
After this operation, 96.3 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 netcat-openbsd amd64 1.187-1ubuntu0.1 [39.8 kB]
Fetched 39.8 kB in 1s (76.9 kB/s)
Selecting previously unselected package netcat-openbsd.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../netcat-openbsd_1.187-1ubuntu0.1_amd64.deb ...
Unpacking netcat-openbsd (1.187-1ubuntu0.1) ...
Setting up netcat-openbsd (1.187-1ubuntu0.1) ...
update-alternatives: using /bin/nc.openbsd to provide /bin/nc (nc) in auto mode
Processing triggers for man-db (2.8

In [4]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql import streaming

import pandas as pd
from IPython.display import display, clear_output
import seaborn as sns

spark = SparkSession.builder.appName("Streaming").getOrCreate()

In [5]:
lines = spark.readStream.format('socket').option('host', 'localhost').option('port', 9999).load()

words = lines.select(f.explode(f.split(lines.value, " ")).alias("word"))

wordCounts = words.groupBy("word").count()

In [ ]:
query = wordCounts.writeStream.outputMode("complete").format("memory").start()

# query.awaitTermination()

# Twitter App 

In [22]:
import sys
sys.path.append("/content/drive/MyDrive/Study/Big Data/Assignment10")

In [23]:
from TwitterSecrets import twitter_secrets as ts

OUT_PATH = "/content/drive/MyDrive/Study/Big Data/Assignment10/DataStreaming"

QUERY = "euro 2021"

STOP_AFTER = 1000

In [18]:
import json
import tempfile
import requests
import pathlib
from datetime import datetime as dt
from uuid import uuid4
from requests_oauthlib import OAuth1Session

In [33]:
pathlib.Path(OUT_PATH).mkdir(parents = True, exist_ok = True)

query_data = {
    "track": f"#{QUERY}".replace("#", "").lower(), "language": "en"
}

twitter = OAuth1Session(
    client_key = ts.CONSUMER_KEY,
    client_secret = ts.CONSUMER_SECRET,
    resource_owner_key = ts.ACCESS_TOKEN,
    resource_owner_secret = ts.ACCESS_SECRET,
)

url = "https://stream.twitter.com/1/1/statuses/filter.json"
query_url = f"{url}?{'&'.join([f'{k}={v}' for k, v in query_data.items()])}"

print(f'STREAMING {STOP_AFTER} TWEETS')

with twitter.get(query_url, stream = True) as response:
  for i, raw_tweet in enumerate(response.iter_lines()):
    if i == STOP_AFTER:
      break
    try:
      tweet = json.loads(raw_tweet)
      print(f"{i+1}/{STOP_AFTER}: {tweet['user']['screen_name']} @ {tweet['created_at']}: {tweet['text']}\n")
    except (json.JSONDecodeError, KeyError) as err:
      print("ERROR")
      continue
    with pathlib.Path(OUT_PATH) / f"{dt.now().timestamp()}_{uuid4()}.json" as F:
      F.write_bytes(raw_tweet)

STREAMING 1000 TWEETS
